### Makes width larger for Jupyter notebook

In [1]:
from IPython.core.display import HTML,Markdown

HTML("""
<style>
.container { width:100% !important; }
</style>
""")

# Goal of Video:
* Introduce dash 
* Create a simple Yahoo finance dashboard that we will great improve in subsequent videos and eventually turn into a website

### Dash Tutorial
* Make sure you have latest version of dash so it will work in Jupyter
    - !pip install --upgrade dash
* Strong Open source + Enterprise Support
* Limitation of JupyterDash is multipages: https://dash.plotly.com/dash-in-jupyter
* Examples:
    - https://plotly.com/examples/
* App Layout
* Dash Componenets
    - Dash HTML Components (html)
        - i.e. H1 + div
    - Dash Core Components (dcc)
        - i.e. Graph: dcc.graph
    - Dash DataTable (dash_table)
        - dash_table.DataTable
* Callbacks
#### NOTE CAN VIEW ON localhost:8050

- **`<h1>` Tag:**
  - Stands for "Heading 1."
  - Main title of the content.
- **`<div>` Tag:**
  - Defines a section/division.
  - Container for HTML elements.
  - No inherent semantic meaning.
  - Groups elements for CSS/JS.


### App Layout
* Some html element like a div..
* Div has children which are its elements

In [2]:
from dash import html,Dash
app = Dash()

app.layout = html.Div(children=[html.H1("Trade Mamba"),
                                html.H2("Select a Ticker")
                               ])

app.run()

### Let's add a dropdown to select a stock 
* Use id to uniquely identify the component which is needed for callbacks

In [3]:
from dash import dcc
symbol_list = ['TSLA','NVDA','AMD','META','RIVN','BABA','SPY','QQQ']

app.layout = html.Div(children=[html.H1("Trade Mamba"),
                                html.H2("Select a Ticker"),
                                dcc.Dropdown(options=symbol_list,value='SPY',id='stock_picker')
                               ])
app.run()

### Let's add a dummy graph
* We will fill in the graph later via the callback

In [4]:
from dash import dcc
import plotly.express as px


app.layout = html.Div(children=[html.H1("Trade Mamba"),
                                html.H2("Select a Ticker"),
                                dcc.Dropdown(options=symbol_list,value='SPY',id='stock_picker'),
                                dcc.Graph(id='stock-plot')
                               ])
app.run()

### Let's add dummy data table
* We will fill in the table later via the callback

In [5]:
from dash import dash_table

app.layout = html.Div(children=[html.H1("Trade Mamba"),
                                html.H2("Select a Ticker"),
                                dcc.Dropdown(options=symbol_list,value='SPY',id='stock_picker'),
                                dcc.Graph(id='stock_plot'),
                                dash_table.DataTable(id='data_table',page_size=20,)
                               ])
app.run()

##### Asside: Datable example

In [6]:

from dash import Dash, dash_table
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/solar.csv')

app.layout = dash_table.DataTable(data=df.to_dict('records'), columns=[{"name": i, "id": i} for i in df.columns])

app.run()


### Let's add some callbacks to make this make more sense

##### Assside on getting data from Yahoo Finance with yfinance

In [7]:
import yfinance as yf
def get_data_from_yfinance(ticker):
    data = yf.download(ticker, period='max', auto_adjust=True)
    data.reset_index(inplace=True)
    return data
get_data_from_yfinance('TSLA')

[*********************100%%**********************]  1 of 1 completed


,Date,Open,High,Low,Close,Volume
0,2010-06-29,1.266667,1.666667,1.169333,1.592667,281494500
1,2010-06-30,1.719333,2.028000,1.553333,1.588667,257806500
2,2010-07-01,1.666667,1.728000,1.351333,1.464000,123282000
3,2010-07-02,1.533333,1.540000,1.247333,1.280000,77097000
4,2010-07-06,1.333333,1.333333,1.055333,1.074000,103003500
...,...,...,...,...,...,...
3510,2024-06-10,176.059998,178.570007,173.169998,173.789993,50869700
3511,2024-06-11,173.919998,174.750000,167.410004,170.660004,64761900
3512,2024-06-12,171.119995,180.550003,169.800003,177.289993,90389400
3513,2024-06-13,188.389999,191.080002,181.229996,182.470001,118984100


### Callbacks
* Use @app.callback 
* Must be defined after defining app=Dash()
* Input: List of Input or Single Input
* Output: List of Output or Single Output
* Callback function below it
* component_id: tells dash what element to work on via the "id" field
* component_property: what property type we are changing

In [14]:
from dash.dependencies import Output, Input

app = Dash()

@app.callback(
    [Output(component_id='stock_plot',component_property='figure'),
     Output(component_id='data_table',component_property='data'),
     Output(component_id='data_table',component_property='columns')],
    Input(component_id='stock_picker',component_property='value')
)
def update_graph(ticker):
    df = get_data_from_yfinance(ticker)
    fig = px.line(data_frame=df,x='Date',y='Close')
    df.sort_values('Date',ascending=False,inplace=True)
    columns=[{"name": i, "id": i} for i in df.columns]
    data = df.to_dict('records')
    return fig,data,columns
    
app.layout = html.Div(children=[html.H1("Trade Mamba"),
                                html.H2("Select a Ticker"),
                                dcc.Dropdown(options=symbol_list,value='SPY',id='stock_picker'),
                                dcc.Graph(id='stock_plot'),
                                dash_table.DataTable(id='data_table',page_size=20,)
                               ])

app.run()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


### Let's make tabs one for the chart and one for 
* Use dcc.Tabs
* label is tab's name

In [16]:
tabs = dcc.Tabs(children=
               [
                   dcc.Tab(label='Chart',children=dcc.Graph(id='stock_plot')),
                   dcc.Tab(label='Table',children=dash_table.DataTable(id='data_table',page_size=20,))
                   
               ]
               )
app.layout = html.Div(children=[html.H1("Trade Mamba"),
                                html.H2("Select a Ticker"),
                                dcc.Dropdown(options=symbol_list,value='SPY',id='stock_picker'),
                                tabs
                               ])

app.run()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


### Full App with Style and Color
* label is usually text label
* style is where we add the style. We can add it to any dash component like core and html and containers and individual elements

In [18]:
from dash import html,Dash,dcc,dash_table
from dash.dependencies import Input, Output
import plotly.express as px
import yfinance as yf

# Assuming get_data_from_yfinance and symbol_list are defined elsewhere
# from some_module import get_data_from_yfinance, symbol_list
symbol_list = ['TSLA', 'NVDA', 'AMD', 'META', 'BABA', 'SPY', 'QQQ']

app = Dash()

@app.callback(
    [Output(component_id='stock_plot', component_property='figure'),
     Output(component_id='data_table', component_property='data')],
    [Input(component_id='ticker_selection', component_property='value')]
)
def update_graph(ticker):
    dff = get_data_from_yfinance(ticker)
    fig = px.line(dff, x='Date', y='Close')
    fig.update_layout(
        plot_bgcolor='#D1C4E9',  # Light purplish background for the plot area
        paper_bgcolor='#E6B0AA',  # Light reddish-purple background for the surrounding area
        font=dict(color='#4B0082')  # Optional: Adjust the font color for better readability
    )
    dff = dff.sort_values('Date', ascending=False)
    columns = [{"name": i, "id": i} for i in dff.columns]
    data = dff.to_dict('records')
    return fig, data

def get_data_from_yfinance(ticker):
    data = yf.download(ticker, period='max', auto_adjust=True)
    data.reset_index(inplace=True)
    return data


background_style = {
    'backgroundColor': '#A1887F',  # Soft purplish background color
    'padding': '20px'  # Padding around the content
}

tab_style = {
    'backgroundColor': '#D1C4E9',  # Soft reddish-purple background color
    'padding': '20px',  # Padding around the tab content
    'border': 'none'
}

selected_tab_style = {
    'backgroundColor': '#E6B0AA',  # Soft purplish background color for selected tab
    'padding': '20px',  # Padding around the tab content
    'border': 'none'
}

data_table_style = {
    'backgroundColor': '#E6B0AA'  # Soft reddish-purple background color for the table
}

dropdown_style = {
    'backgroundColor': '#F8BBD0',  # Reddish-brown background color 
    'color': '#4B2E2A',  # Text color
    'border': 'none',  # No border
    'padding': '10px',  # Padding inside the dropdown
    'font-size': '16px'  # Font size
}

graph_style = {'backgroundColor': '#E0F7FA'}
    

tabs = dcc.Tabs(children=[ 
    dcc.Tab(label='Chart', children=dcc.Graph(id='stock_plot', style=graph_style), style=tab_style, selected_style=selected_tab_style),
    dcc.Tab(
        label='Table', 
        children=dash_table.DataTable(
            id='data_table', 
            page_size=10, 
            style_table={'backgroundColor': '#E6B0AA'},  # Table background color
            style_data={'backgroundColor': '#E6B0AA', 'color': '#4B0082'},  # Data cell background and text color
            style_header={'backgroundColor': '#E0F7FA', 'color': '#4B0082'}  # Header cell background and text color
        ), 
        style=tab_style, selected_style=selected_tab_style
    )
])

app.layout = html.Div(style=background_style, children=[
    html.H1("Yahoo Finance App", style={'text-align': 'center'}),
    html.Div(children=[
        html.H2("Select a stock"),
        dcc.Dropdown(options=[{'label': sym, 'value': sym} for sym in symbol_list], id='ticker_selection', value='QQQ',style=dropdown_style),
        tabs
    ])

])

app.run()


[*********************100%%**********************]  1 of 1 completed
